# Create interactive map of the Netherlands 
This.,.,.,

In [1]:
import pandas as pd
import json
import folium
from folium import IFrame
import os
from branca.colormap import linear
import matplotlib.pyplot as plt
import mpld3
from mpld3.plugins import LineLabelTooltip, connect
%matplotlib inline
filepath = 'C:/Users/LIONEL/Google Drive/Advanced System Dynamics/Project Wouter-Vincent/'
#filepath = 'C:/Users/Dirk/Google Drive/Dirk/TU/EPA 1/Advanced System Dynamics/Advanced System Dynamics/Project Wouter-Vincent/'

# Load data



* Load map with municipalities data
* Load ouput of vensmin model (for now only final output is given)

In [2]:
#load json file
nl_municipalities = filepath + 'json/Gemeentegrenzen_NL.json'
geo_json_data = json.load(open(nl_municipalities))


#load model results
nl_municipalities_xlsx = filepath +  'data/20170531 Vluchteling model_result.xlsx'
municipality_data = pd.read_excel(nl_municipalities_xlsx,'output')

woz_xlsx = filepath +  'Excel data/WOZ_waarde_per_gemeente_2016.xlsx'
woz_data = pd.read_excel(woz_xlsx,'Gemiddelde_woningprijs')

inwoners_xlsx = filepath + 'Excel data/Inwonersaantallen.xlsx'
inwoners_data = pd.read_excel(inwoners_xlsx,'Sheet1')

twitter_historic_xlsx = filepath + 'Excel data/twitter_activiteit_totaal.xlsx' 
twitter_historic_data= pd.read_excel(twitter_historic_xlsx,'totaal')

output1_dict = municipality_data.set_index('GemeenteCode')['output1'] # instroom model data jan_17 
output2_dict = municipality_data.set_index('GemeenteCode')['output2'] # incidenten model data

#chose one of the 2 output 3
#output3_dict = municipality_data.set_index('GemeenteCode')['output3'] # twitter model data 
output3_dict = twitter_historic_data.set_index('Gemeentecode')['Totaal']
output3_dict_ordinaal = twitter_historic_data.set_index('Gemeentecode')['Ordinaal']#titter data based on historic data excel 

output4_dict = municipality_data.set_index('GemeenteCode')['output4'] # verkiezingsdata model data
output5_dict=woz_data.set_index('Gemeentecode')['WOZ'] #WOZ based on excel
output6_dict=inwoners_data.set_index('Gemeentecode')['Inwoners'] #WOZ based on excel

print("Test print of gemeente Meierijstad")
print(output1_dict['GM1948']) 
print(output2_dict['GM1948'])
print(output3_dict['GM1948'])
print(output4_dict['GM1948'])
print(output5_dict['GM1948'])
print(output6_dict['GM1948'])

Test print of gemeente Meierijstad
0
0
0
7
256000
79869


# Color patterns
* create collor patterns of output variables

In [3]:
#all possible patterns
# 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu',
# 'YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd

#colormap1=output1
colormap1 = linear.YlGn.scale(
    municipality_data.output1.min(),
    municipality_data.output1.max())

#colormap2=output2
colormap2 = linear.BuPu.scale(
    municipality_data.output2.min(),
    municipality_data.output2.max())

#has to be changed if model data or static excel data is used
#colormap3=output3
colormap3 = linear.YlOrRd.scale(
    twitter_historic_data.Ordinaal.min(),
    twitter_historic_data.Ordinaal.max())
    #municipality_data.output3.min(),
    #municipality_data.output3.max())

#colormap4=output4
colormap4 = linear.YlOrBr.scale(
    municipality_data.output4.min(),
    municipality_data.output4.max())

#colormap5=output5
colormap5 = linear.OrRd.scale(
    woz_data.WOZ.min(),
    woz_data.WOZ.max())

#colormap6=output6
colormap6 = linear.YlOrBr.scale(
    inwoners_data.Inwoners.min(),
    inwoners_data.Inwoners.max())


In [4]:
print("colarmap of output1")
colormap1

colarmap of output1


In [5]:
print("colarmap of output2")
colormap2

colarmap of output2


In [6]:
print("colarmap of output3")
colormap3

colarmap of output3


In [7]:
print("colarmap of output4")
colormap4

colarmap of output4


In [8]:
print("colarmap of output5")
colormap5

colarmap of output5


In [9]:
print("colarmap of output6")
colormap6

colarmap of output6


# Popup content
* Create per layer a different popup.
* Read Vensim output graphs.
* Create html page based on municipality and layer.

In [10]:
#gather information per municipality
dict_child_output1={}
dict_child_output2={}
dict_child_output3={}
dict_child_output4={}
dict_child_output5={}
dict_child_output6={}
#make tile of the coordinates per municipality

#####################output1 instroom jan17######################
for feature in geo_json_data['features']:
    #print(feature['properties']['name'])
    child1 = folium.GeoJson(
        data={
            "type": feature['geometry']['type'],
            "coordinates":  feature['geometry']['coordinates']},
        name=feature['properties']['gemeentena'],
        style_function = lambda feature: dict(color='grey', weight= 0),
        control=False,
    )  
    #create per tile an iFrame with its name + output variables
    text = "Gemeente: {}<br> Instroom: {}<br> Incidenten: {} <br> Social_data: {} <br> Verkiezing_data: {} <br> WOZ: {} <br> Inwoners: {} <br> ".format(
        feature['properties']['gemeentena'],
        output1_dict[feature['properties']['code']],
        output2_dict[feature['properties']['code']],
        output3_dict[feature['properties']['code']],
        output4_dict[feature['properties']['code']],
        output5_dict[feature['properties']['code']],
        output6_dict[feature['properties']['code']])
    #make dict with key tile coordinates and values its name + output variables 
    dict_child_output1[child1]=text

########################outoput 2 (incidenten) ###########################################33
for feature in geo_json_data['features']:
    #print(feature['properties']['name'])
    child2 = folium.GeoJson(
        data={
            "type": feature['geometry']['type'],
            "coordinates":  feature['geometry']['coordinates']},
        name=feature['properties']['gemeentena'],
        style_function = lambda feature: dict(color='white', weight= 0),
        control=False,
    )  
    #create per tile an iFrame with its name + output variables
    figname=filepath + "HTML_graphs/Social2/socialmedia_"+feature['properties']['code']+".html"
    with open(figname, 'r') as f:
             html = """<head>
                    <h2><b>Gemeente {}
                    </head></b></h2>
                    <body>
                    Aantal vluchtelingen in gemeente: <i>{}</i><br>
                    Aantal Incidenten per maand: <i>{}</i><br>
                    Aantal socialmedia berichten over vluchtelingen laatste 6 maanden: <i>{}</i><br>
                    Grootst politieke partij: <i>{}</i><br>
                    ....<br>

                    </body>
                    {}.""".format(
                            feature['properties']['gemeentena'],
                            output1_dict[feature['properties']['code']],
                            output2_dict[feature['properties']['code']],
                            output3_dict[feature['properties']['code']],
                            output4_dict[feature['properties']['code']],
                            f.read())
    #make dict with key tile coordinates and values its name + output variables 
    dict_child_output2[child2]=html
    
    
###################output 3 (Social)###############################
for feature in geo_json_data['features']:
    #print(feature['properties']['name'])
    child3 = folium.GeoJson(
        data={
            "type": feature['geometry']['type'],
            "coordinates":  feature['geometry']['coordinates']},
        name=feature['properties']['gemeentena'],
        style_function = lambda feature: dict(color='white', weight= 0),
        control=False,
    )  
    #create per tile an iFrame with its name + output variables
    figname= filepath + "HTML_graphs/Social2/socialmedia_"+feature['properties']['code']+".html"
    with open(figname, 'r') as f:
             html = """<head>
                    <h2><b>Gemeente {}
                    </head></b></h2>
                    <body>
                    Aantal vluchtelingen in gemeente: <i>{}</i><br>
                    Aantal Incidenten per maand: <i>{}</i><br>
                    Aantal socialmedia berichten over vluchtelingen laatste 6 maanden: <i>{}</i><br>
                    Grootst politieke partij: <i>{}</i><br>
                    ....<br>

                    </body>
                    {}.""".format(
                            feature['properties']['gemeentena'],
                            output1_dict[feature['properties']['code']],
                            output2_dict[feature['properties']['code']],
                            output3_dict[feature['properties']['code']],
                            output4_dict[feature['properties']['code']],
                            f.read())
    #make dict with key tile coordinates and values its name + output variables 
    dict_child_output3[child3]=html

################output 4##################################

for feature in geo_json_data['features']:
    #print(feature['properties']['name'])
    child4 = folium.GeoJson(
        data={
            "type": feature['geometry']['type'],
            "coordinates":  feature['geometry']['coordinates']},
        name=feature['properties']['gemeentena'],
        style_function = lambda feature: dict(color='grey', weight= 0),
        control=False,
    )  
    #create per tile an iFrame with its name + output variables
    text = "Gemeente: {}<br> Instroom: {}<br> Incidenten: {} <br> Social_data: {} <br> Verkiezing_data: {} <br> WOZ: {} <br> Inwoners: {} <br> ".format(
        feature['properties']['gemeentena'],
        output1_dict[feature['properties']['code']],
        output2_dict[feature['properties']['code']],
        output3_dict[feature['properties']['code']],
        output4_dict[feature['properties']['code']],
        output5_dict[feature['properties']['code']],
        output6_dict[feature['properties']['code']])
    #make dict with key tile coordinates and values its name + output variables 
    dict_child_output4[child4]=text

#####OUTPUT 5########
for feature in geo_json_data['features']:
    #print(feature['properties']['name'])
    child5 = folium.GeoJson(
        data={
            "type": feature['geometry']['type'],
            "coordinates":  feature['geometry']['coordinates']},
        name=feature['properties']['gemeentena'],
        style_function = lambda feature: dict(color='grey', weight= 0),
        control=False,
    )  
    #create per tile an iFrame with its name + output variables
    text = "Gemeente: {}<br> Instroom: {}<br> Incidenten: {} <br> Social_data: {} <br> Verkiezing_data: {} <br> WOZ: {} <br> Inwoners: {} <br> ".format(
        feature['properties']['gemeentena'],
        output1_dict[feature['properties']['code']],
        output2_dict[feature['properties']['code']],
        output3_dict[feature['properties']['code']],
        output4_dict[feature['properties']['code']],
        output5_dict[feature['properties']['code']],
        output6_dict[feature['properties']['code']])
    #make dict with key tile coordinates and values its name + output variables 
    dict_child_output5[child5]=text

#####OUTPUT 6########    
for feature in geo_json_data['features']:
    #print(feature['properties']['name'])
    child6 = folium.GeoJson(
        data={
            "type": feature['geometry']['type'],
            "coordinates":  feature['geometry']['coordinates']},
        name=feature['properties']['gemeentena'],
        style_function = lambda feature: dict(color='grey', weight= 0),
        control=False,
    )  
    #create per tile an iFrame with its name + output variables
    text = "Gemeente: {}<br> Instroom: {}<br> Incidenten: {} <br> Social_data: {} <br> Verkiezing_data: {} <br> WOZ: {} <br> Inwoners: {} <br> ".format(
        feature['properties']['gemeentena'],
        output1_dict[feature['properties']['code']],
        output2_dict[feature['properties']['code']],
        output3_dict[feature['properties']['code']],
        output4_dict[feature['properties']['code']],
        output5_dict[feature['properties']['code']],
        output6_dict[feature['properties']['code']])
    #make dict with key tile coordinates and values its name + output variables 
    dict_child_output6[child6]=text

In [11]:
print(next (iter (dict_child_output1.keys())))
print(next (iter (dict_child_output1.values())))

Gemeente: Raalte<br> Instroom: 0<br> Incidenten: 4 <br> Social_data: 0 <br> Verkiezing_data: 3 <br> WOZ: 227000 <br> Inwoners: 36907 <br> 


In [12]:
print(next (iter (dict_child_output2.keys())))
print(next (iter (dict_child_output2.values())))

<head>
                    <h2><b>Gemeente Raalte
                    </head></b></h2>
                    <body>
                    Aantal vluchtelingen in gemeente: <i>0</i><br>
                    Aantal Incidenten per maand: <i>4</i><br>
                    Aantal socialmedia berichten over vluchtelingen laatste 6 maanden: <i>0</i><br>
                    Grootst politieke partij: <i>3</i><br>
                    ....<br>

                    </body>
                    

<style>

</style>

<div id="fig_el1086424188822524727306102210"></div>
<script>
function mpld3_load_lib(url, callback){
  var s = document.createElement('script');
  s.src = url;
  s.async = true;
  s.onreadystatechange = s.onload = callback;
  s.onerror = function(){console.warn("failed to load library " + url);};
  document.getElementsByTagName("head")[0].appendChild(s);
}

if(typeof(mpld3) !== "undefined" && mpld3._mpld3IsLoaded){
   // already loaded: just create the figure
   !function(mpld3){
       
      

In [13]:
print(next (iter (dict_child_output3.keys())))
print(next (iter (dict_child_output3.values())))

<head>
                    <h2><b>Gemeente Raalte
                    </head></b></h2>
                    <body>
                    Aantal vluchtelingen in gemeente: <i>0</i><br>
                    Aantal Incidenten per maand: <i>4</i><br>
                    Aantal socialmedia berichten over vluchtelingen laatste 6 maanden: <i>0</i><br>
                    Grootst politieke partij: <i>3</i><br>
                    ....<br>

                    </body>
                    

<style>

</style>

<div id="fig_el1086424188822524727306102210"></div>
<script>
function mpld3_load_lib(url, callback){
  var s = document.createElement('script');
  s.src = url;
  s.async = true;
  s.onreadystatechange = s.onload = callback;
  s.onerror = function(){console.warn("failed to load library " + url);};
  document.getElementsByTagName("head")[0].appendChild(s);
}

if(typeof(mpld3) !== "undefined" && mpld3._mpld3IsLoaded){
   // already loaded: just create the figure
   !function(mpld3){
       
      

In [14]:
print(next (iter (dict_child_output4.keys())))
print(next (iter (dict_child_output4.values())))

Gemeente: Raalte<br> Instroom: 0<br> Incidenten: 4 <br> Social_data: 0 <br> Verkiezing_data: 3 <br> WOZ: 227000 <br> Inwoners: 36907 <br> 


In [15]:
print(next (iter (dict_child_output5.keys())))
print(next (iter (dict_child_output5.values())))

Gemeente: Raalte<br> Instroom: 0<br> Incidenten: 4 <br> Social_data: 0 <br> Verkiezing_data: 3 <br> WOZ: 227000 <br> Inwoners: 36907 <br> 


In [16]:
print(next (iter (dict_child_output6.keys())))
print(next (iter (dict_child_output6.values())))

Gemeente: Raalte<br> Instroom: 0<br> Incidenten: 4 <br> Social_data: 0 <br> Verkiezing_data: 3 <br> WOZ: 227000 <br> Inwoners: 36907 <br> 


# Create map
* Create map and 4 empty layers
* Fill different layers based on color patterns
* Add popups to layer
* Add layers to map

In [17]:
#create map NL
m1 = folium.Map(location=[52, 5], zoom_start=8)
#Create 4 empty layers
fg_output1=folium.FeatureGroup(name="Instroom")
fg_output2=folium.FeatureGroup(name="Incidenten")
fg_output3=folium.FeatureGroup(name="Social")
fg_output4=folium.FeatureGroup(name="Verkiezingen")
fg_output5=folium.FeatureGroup(name="WOZ")
fg_output6=folium.FeatureGroup(name="Inwoners")

In [18]:
#Color layers based on color patterns

#Color output1 layer
fg_output1.add_child(folium.GeoJson(
    geo_json_data,
    name='output1_dict',
    style_function=lambda feature: {
        'fillColor': colormap1(output1_dict[feature['properties']['code']]),
        'color': 'blue',
        'weight': 5,
        'dashArray': '5, 5',
        'fillOpacity': 0.5,
    },
))
#Color output2 layer
fg_output2.add_child(folium.GeoJson(
    geo_json_data,
    name='output2_dict',
    style_function=lambda feature: {
        'fillColor': colormap2(output2_dict[feature['properties']['code']]),
        'color': 'blue',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    },
))
#Color output3 layer
fg_output3.add_child(folium.GeoJson(
    geo_json_data,
    name='output3_dict',
    style_function=lambda feature: {
        'fillColor': 'black' if output3_dict_ordinaal[feature['properties']['code']] is None else colormap3(output3_dict_ordinaal[feature['properties']['code']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.5,
    },
))
#Color output4 layer
fg_output4.add_child(folium.GeoJson(
    geo_json_data,
    name='output4_dict',
    style_function=lambda feature: {
        'fillColor': colormap4(output4_dict[feature['properties']['code']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    },
))

#Color output5 layer
fg_output5.add_child(folium.GeoJson(
    geo_json_data,
    name='output5_dict',
    style_function=lambda feature: {
        'fillColor': colormap5(output5_dict[feature['properties']['code']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    },
))

#Color output6 layer
fg_output6.add_child(folium.GeoJson(
    geo_json_data,
    name='output6_dict',
    style_function=lambda feature: {
        'fillColor': colormap6(output6_dict[feature['properties']['code']]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    },
))

In [19]:
#Add popups to layers
for i in dict_child_output1:
    i.add_child(folium.Popup(IFrame(dict_child_output1[i], width=250, height=100), max_width= 550))
    i.add_to(fg_output1)
for i in dict_child_output2:
    i.add_child(folium.Popup(IFrame(dict_child_output2[i], width=550, height=550), max_width= 550))
    i.add_to(fg_output2)
for i in dict_child_output3:
    i.add_child(folium.Popup(IFrame(dict_child_output3[i], width=550, height=550), max_width= 550))
    i.add_to(fg_output3)
for i in dict_child_output4:
    i.add_child(folium.Popup(IFrame(dict_child_output4[i], width=250, height=100), max_width= 550))
    i.add_to(fg_output4)
for i in dict_child_output5:
    i.add_child(folium.Popup(IFrame(dict_child_output5[i], width=250, height=100), max_width= 550))
    i.add_to(fg_output5)
for i in dict_child_output6:
    i.add_child(folium.Popup(IFrame(dict_child_output6[i], width=250, height=100), max_width= 550))
    i.add_to(fg_output6)

In [20]:
#add all layers to the map
fg_output1.add_to(m1)
fg_output2.add_to(m1)
fg_output3.add_to(m1)
fg_output4.add_to(m1)
fg_output5.add_to(m1)
fg_output6.add_to(m1)

In [21]:
#make a layercontrol button
folium.LayerControl(position='topright',collapsed=False).add_to(m1)
#save map as html file
m1.save(os.path.join(filepath + "results",'Vluchtelingenkaart3.html'))

In [22]:
## TEST

In [23]:
## TEST?